## Auto-correct incorrect reference key

In this notebook we analyse the citations of articles and try to match them against other articles in the database. The reference keys of the citations and articles are not always consistent. Unmatched citations are then matched against articles based only on author and year information. This results in a one-to-many match. From there, the journal titles of citation and potential articles in the database are scored using natural-language processing techniques. An alternate title is also used. The score, together with the article names are then used to either match the citation to one or non-articles, depending on the score.

The articles and their citations were extracted using:

* `bibliometrix_report_carp_grp.Rmd`

In [226]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

First we read the articles. Each article's citation is in the `CR` field, and a unique key for the article is in the `Unnamed: 0` field.

In [227]:
d_all = pd.read_csv('../data/arp_grp_2010_2019_references.csv')
d_carps = pd.read_csv('../data/arp_grp_2010_2019_references_limited_comments.csv')
d_carps_key = d_carps.loc[d_carps['CARP (Yes/No)'] == 1]['Unnamed: 0']
d = d_all[d_all['Unnamed: 0'].isin(d_carps_key)]
print(len(d_all), len(d_carps), len(d))

237 237 192


The `split_citation` function split the article key into author, year and journal. A secondary title is also created based on `SE` key. The `split_citation2` function splits a reference into the same info.

In [228]:
def split_citation(line):
    d = line.split(',')
    author = d[0].strip()
    year = d[1].strip()
    journal = ','.join(d[2:])
    author_year = ','.join(d[:2]).strip()
    return([author, year, journal, author_year, line])

def split_citation2(line_full):    
    author = []
    year = []
    journal = []
    author_year = []
    full = []
    if line_full is np.nan: # some articles don't have any citations
        print('no references')
    else: # the rest are split and the info extracted
        ref = line_full.split(';')
        for  r in ref:
            d = r.split(',')
            n_entries = len(d) - 1
            author.append(d[min([0, n_entries])].strip())
            year.append(d[min([1, n_entries])].strip())
            journal.append(','.join(d[min([2, n_entries]):min([3, n_entries])]))
            author_year.append(','.join(d[:min([2, n_entries])]).strip())
            full.append(','.join(d[:min([3, n_entries])]))
    
    # Cited article info are given `_cite` extension
    citation_df = pd.DataFrame.from_dict({'author_cite': author, 
                                          'year_cite': year, 
                                          'journal_cite': journal, 
                                          'author_year_cite': author_year,
                                          'full_cite': full})
    return(citation_df)

author_info = d['Unnamed: 0'].apply(split_citation)
author_info = author_info.apply(pd.Series)
author_info = author_info.rename(columns = {0: 'author', 1: 'year', 2: 'journal', 3: 'author_year', 4: 'full'})
author_info['alt_journal'] = d['SE']
author_info

,author,year,journal,author_year,full,alt_journal
1,WILLEMSE EJ,2019,COMPUT OPER RES,"WILLEMSE EJ, 2019","WILLEMSE EJ, 2019, COMPUT OPER RES",NaN
2,ARAKAKI RK,2019,COMPUT OPER RES,"ARAKAKI RK, 2019","ARAKAKI RK, 2019, COMPUT OPER RES",NaN
3,DE ARMAS J,2019,ANN OPER RES,"DE ARMAS J, 2019","DE ARMAS J, 2019, ANN OPER RES",NaN
5,TFAILI S,2019,RAIRO-OPER RES,"TFAILI S, 2019","TFAILI S, 2019, RAIRO-OPER RES",NaN
6,MOFID-NAKHAEE E,2019,WASTE MANAGE RES,"MOFID-NAKHAEE E, 2019","MOFID-NAKHAEE E, 2019, WASTE MANAGE RES",NaN
7,FENG L,2019,EVOLUTIONARY AND SWARM INTELLIGENCE ALGORITHMS,"FENG L, 2019","FENG L, 2019, EVOLUTIONARY AND SWARM INTELLIGE...",STUDIES IN COMPUTATIONAL INTELLIGENCE
9,LI M,2018,COMPUT IND ENG,"LI M, 2018","LI M, 2018, COMPUT IND ENG",NaN
10,TIRKOLAEE EB,2018,WASTE MANAGE,"TIRKOLAEE EB, 2018","TIRKOLAEE EB, 2018, WASTE MANAGE",NaN
11,SHANG R,2018,NAT COMPUT,"SHANG R, 2018","SHANG R, 2018, NAT COMPUT",NaN
12,CIANCIO C,2018,EUR J OPER RES,"CIANCIO C, 2018","CIANCIO C, 2018, EUR J OPER RES",NaN


Next we extract all the citations per article, and match the citations against the articles in the database. Citations that are not fully matched but matched on author-year are then stored for further analysis. These citations can potential still be matched to those in the database.

In [229]:
partial_citations = []
for i, citation in enumerate(d['CR'].values): # each article's citations are extracted.
    print('\n', i, author_info.iloc[i]['full'])
    author_citation = split_citation2(citation) 
    
    # citations are left matched against articles
    not_fully_cited = author_citation.merge(author_info, how='left', left_on=['full_cite'], right_on=['full'])
    # citations without matches have NA values for author_y
    not_fully_cited = not_fully_cited.loc[pd.isnull(not_fully_cited['author'])]
    not_fully_cited = not_fully_cited.drop(columns = ['author', 'year', 'journal', 'author_year', 'full', 'alt_journal'])
    
    # those without full matches are matched using only author and year
    author_cited = not_fully_cited.merge(author_info, how='inner', left_on=['author_year_cite'], right_on=['author_year'])    
    print('Potential citations not immediately matched:', len(author_cited))
    partial_citations.append(author_cited)


 0 WILLEMSE EJ, 2019, COMPUT OPER RES
Potential citations not immediately matched: 4

 1 ARAKAKI RK, 2019, COMPUT OPER RES
Potential citations not immediately matched: 6

 2 DE ARMAS J, 2019, ANN OPER RES
Potential citations not immediately matched: 9

 3 TFAILI S, 2019, RAIRO-OPER RES
Potential citations not immediately matched: 1

 4 MOFID-NAKHAEE E, 2019, WASTE MANAGE RES
Potential citations not immediately matched: 9

 5 FENG L, 2019, EVOLUTIONARY AND SWARM INTELLIGENCE ALGORITHMS
Potential citations not immediately matched: 5

 6 LI M, 2018, COMPUT IND ENG
Potential citations not immediately matched: 1

 7 TIRKOLAEE EB, 2018, WASTE MANAGE
Potential citations not immediately matched: 1

 8 SHANG R, 2018, NAT COMPUT
Potential citations not immediately matched: 17

 9 CIANCIO C, 2018, EUR J OPER RES
Potential citations not immediately matched: 3

 10 TIRKOLAEE EB, 2018, SUSTAINABILITY
Potential citations not immediately matched: 5

 11 SHANG R, 2018, MEMET COMPUT
Potential citations

Potential citations not immediately matched: 1

 115 MEI Y, 2011, IEEE TRANS EVOL COMPUT
Potential citations not immediately matched: 4

 116 XU H, 2011, EXPERT SYST APPL
Potential citations not immediately matched: 1

 117 TAGMOUTI M, 2011, TRANSP RES PT C-EMERG TECHNOL
Potential citations not immediately matched: 1

 118 MARTINELLI R, 2011, EXPERIMENTAL ALGORITHMS
Potential citations not immediately matched: 3

 119 TAGMOUTI M, 2010, COMPUT IND ENG
Potential citations not immediately matched: 4

 120 ARCHETTI C, 2010, COMPUT OPER RES
Potential citations not immediately matched: 1

 121 VANSTEENWEGEN P, 2010, COMPUT OPER RES
Potential citations not immediately matched: 2

 122 MUYLDERMANS L, 2010, COMPUT OPER RES
Potential citations not immediately matched: 5

 123 CORBERAN A, 2010, NETWORKS
Potential citations not immediately matched: 12

 124 AMAYA CA, 2010, J OPER RES SOC
Potential citations not immediately matched: 1

 125 XING L, 2010, IEEE TRANS EVOL COMPUT
Potential citations n

In [230]:
partial_citations_all = pd.concat(partial_citations)
print(partial_citations_all.columns)
print('\nTotal number of citations partially matched', len(partial_citations_all))

Index(['author_cite', 'year_cite', 'journal_cite', 'author_year_cite',
       'full_cite', 'author', 'year', 'journal', 'author_year', 'full',
       'alt_journal'],
      dtype='object')

Total number of citations partially matched 692


A number of these are duplicate records, meaning multiple articles cited the same partially matched source, which can be removed:

In [231]:
partial_citations_uniq = partial_citations_all.drop_duplicates()
print('\nTotal number of citations partially matched and unique', len(partial_citations_uniq))


Total number of citations partially matched and unique 115


Now we use natural-language-processing to score the match of the cited article against the partially matched available articles: 

In [232]:
def fuzzy_match(x, y):
    if not pd.isna(y):
        return fuzz.token_set_ratio(x, y)
    else:
        return 0
    
partial_citations_uniq['journal_match_score'] = partial_citations_uniq.apply(lambda x: fuzzy_match(x['journal_cite'], x['journal']), axis=1)
partial_citations_uniq['alt_journal_match_score'] = partial_citations_uniq.copy().apply(lambda x: fuzzy_match(x['journal_cite'], x['alt_journal']), axis=1)
partial_citations_uniq['max_score'] = partial_citations_uniq[['journal_match_score', 'alt_journal_match_score']].max(1)
partial_citations_uniq['journal_final'] =  partial_citations_uniq['journal']
alt_journal_better = (partial_citations_uniq['max_score'] > 0) & (partial_citations_uniq['alt_journal_match_score'] > 0)
partial_citations_uniq.loc[alt_journal_better, 'journal_final'] = partial_citations_uniq.loc[alt_journal_better, 'alt_journal']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [233]:
partial_citations_uniq.sort_values(by=['max_score', 'full_cite'], ascending=False, inplace=True)
parts = partial_citations_uniq[['year_cite', 'journal_cite', 'journal_final','max_score', 'journal']]
parts

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,year_cite,journal_cite,journal_final,max_score,journal
1,2006,OPER RES,COMPUT OPER RES,100,COMPUT OPER RES
6,2016,PROCESS SAF ENVIRON,PROCESS SAF ENVIRON PROTECT,100,PROCESS SAF ENVIRON PROTECT
0,2009,CIE: 2009 INTERNATIONAL CONFERENCE ON COMPUTE...,CIE: 2009 INTERNATIONAL CONFERENCE ON COMPUTE...,100,CIE: 2009 INTERNATIONAL CONFERENCE ON COMPUTE...
1,2005,GRAPH THEORY,"GRAPH THEORY, COMBINATORICS AND ALGORITHMS: I...",100,"GRAPH THEORY, COMBINATORICS AND ALGORITHMS: I..."
4,2013,DISCRETE APPL MATH,DISCRET APPL MATH,97,DISCRET APPL MATH
1,2014,CAN J CIVIL ENG,CAN J CIV ENG,93,CAN J CIV ENG
6,2012,AUTOMAT CONSTR,AUTOM CONSTR,92,AUTOM CONSTR
0,2008,ADV COMPUTATIONAL IN,STUDIES IN COMPUTATIONAL INTELLIGENCE,89,ADVANCES IN COMPUTATIONAL INTELLIGENCE IN TRA...
1,2008,ADV COMPUTATIONAL IN,STUDIES IN COMPUTATIONAL INTELLIGENCE,89,RECENT ADVANCES IN EVOLUTIONARY COMPUTATION F...
1,2010,TRANSPORT SCI,TRANSP SCI,87,TRANSP SCI


We can then inspect the matches. And we can look at the max score per group:

In [234]:
pd.set_option('display.max_rows', 120)
idx = parts.groupby(['journal_cite', 'year_cite'])['max_score'].transform(max) == parts['max_score']
parts_f = parts[idx]
parts_f.to_csv('../data/alternate_journal_titles.csv', index=False)

With the new journal titles set in `../data/alternate_journal_titles_cleaned.csv` we can replace the citations of articles that have partial matches with those with alternate journal titles.

In [235]:
alt_titles = pd.read_csv('../data/alternate_journal_titles_cleaned.csv').drop_duplicates()
alt_titles['year_cite'] = alt_titles['year_cite'].astype(str)

We repeat the process, this time with the alternate journal titles included:

In [236]:
new_citations = []
print(len(d))
for i, citation in enumerate(d['CR'].values): # each article's citations are extracted.
    print('\n', i, author_info.iloc[i]['full'])
    author_citation = split_citation2(citation)
    
    # we again extract the fully matched references
    fully_cited = author_citation.merge(author_info, how='inner', left_on=['full_cite'], right_on=['full'])
    
    # citations are left matched against articles
    not_fully_cited = author_citation.merge(author_info, how='left', left_on=['full_cite'], right_on=['full'])
    
    # citations without matches have NA values for author_y
    not_fully_cited = not_fully_cited.loc[pd.isnull(not_fully_cited['author'])]
    not_fully_cited = not_fully_cited.drop(columns = ['author', 'year', 'journal', 'author_year', 'full', 'alt_journal'])
    
    # those without full matches are matched using only author and year
    author_cited = not_fully_cited.merge(author_info, how='inner', left_on=['author_year_cite'], right_on=['author_year'])    
    
    # for the rest, their journal_citation is updated to those of `alt_titles` and matched based on journal:
    journal_match =  pd.merge(author_cited, alt_titles, how='inner', left_on=['journal_cite', 'year_cite'], right_on=['journal_cite', 'year_cite'])
    
    # a new list of full citations is created, matching the article's
    
    all_matches = list(fully_cited['full'].values)
    all_2nd_matches = list(journal_match['full'].values)
    citations = all_matches + all_2nd_matches
    
    # and added to a new file `CR_matched`
    new_citations.append(';'.join(citations))
    print('Full citation mathces', len(fully_cited))
    print('Potential citations not immediately matched:', len(author_cited))
    print('Citations matched on alternate journal names:', len(journal_match))
    print('Final total matches', len(citations))

192

 0 WILLEMSE EJ, 2019, COMPUT OPER RES
Full citation mathces 9
Potential citations not immediately matched: 4
Citations matched on alternate journal names: 4
Final total matches 13

 1 ARAKAKI RK, 2019, COMPUT OPER RES
Full citation mathces 13
Potential citations not immediately matched: 6
Citations matched on alternate journal names: 5
Final total matches 18

 2 DE ARMAS J, 2019, ANN OPER RES
Full citation mathces 18
Potential citations not immediately matched: 9
Citations matched on alternate journal names: 7
Final total matches 25

 3 TFAILI S, 2019, RAIRO-OPER RES
Full citation mathces 6
Potential citations not immediately matched: 1
Citations matched on alternate journal names: 1
Final total matches 7

 4 MOFID-NAKHAEE E, 2019, WASTE MANAGE RES
Full citation mathces 14
Potential citations not immediately matched: 9
Citations matched on alternate journal names: 9
Final total matches 23

 5 FENG L, 2019, EVOLUTIONARY AND SWARM INTELLIGENCE ALGORITHMS
Full citation mathces 0
Pote

Full citation mathces 9
Potential citations not immediately matched: 2
Citations matched on alternate journal names: 1
Final total matches 10

 62 ZHANG W, 2015, KNOWLEDGE SCIENCE, ENGINEERING AND MANAGEMENT, KSEM 2015
Full citation mathces 2
Potential citations not immediately matched: 4
Citations matched on alternate journal names: 4
Final total matches 6

 63 FADZLI M, 2015, INTERNATIONAL CONFERENCE ON MATHEMATICS, ENGINEERING AND INDUSTRIAL APPLICATIONS 2014 (ICOMEIA 2014)
Full citation mathces 10
Potential citations not immediately matched: 1
Citations matched on alternate journal names: 1
Final total matches 11

 64 LIU T, 2014, FLEX SERV MANUF J
Full citation mathces 6
Potential citations not immediately matched: 7
Citations matched on alternate journal names: 7
Final total matches 13

 65 BOSCO A, 2014, NETWORKS
Full citation mathces 4
Potential citations not immediately matched: 0
Citations matched on alternate journal names: 0
Final total matches 4

 66 RIQUELME-RODRIGUEZ JP,

Full citation mathces 2
Potential citations not immediately matched: 1
Citations matched on alternate journal names: 1
Final total matches 3

 121 VANSTEENWEGEN P, 2010, COMPUT OPER RES
Full citation mathces 4
Potential citations not immediately matched: 2
Citations matched on alternate journal names: 2
Final total matches 6

 122 MUYLDERMANS L, 2010, COMPUT OPER RES
Full citation mathces 7
Potential citations not immediately matched: 5
Citations matched on alternate journal names: 4
Final total matches 11

 123 CORBERAN A, 2010, NETWORKS
Full citation mathces 23
Potential citations not immediately matched: 12
Citations matched on alternate journal names: 10
Final total matches 33

 124 AMAYA CA, 2010, J OPER RES SOC
Full citation mathces 3
Potential citations not immediately matched: 1
Citations matched on alternate journal names: 1
Final total matches 4

 125 XING L, 2010, IEEE TRANS EVOL COMPUT
Full citation mathces 9
Potential citations not immediately matched: 6
Citations matched 

Final total matches 6

 177 CHU F, 2004, SERVICE SYSTEMS AND SERVICE MANAGEMENT - PROCEEDINGS OF ICSSSM `04, VOLS 1 AND 2
Full citation mathces 2
Potential citations not immediately matched: 3
Citations matched on alternate journal names: 2
Final total matches 4

 178 AMPONSAH SK, 2004, WASTE MANAGE
Full citation mathces 1
Potential citations not immediately matched: 0
Citations matched on alternate journal names: 0
Final total matches 1

 179 MUYLDERMANS L, 2003, J OPER RES SOC
Full citation mathces 1
Potential citations not immediately matched: 1
Citations matched on alternate journal names: 0
Final total matches 1

 180 BEULLENS P, 2003, EUR J OPER RES
Full citation mathces 2
Potential citations not immediately matched: 0
Citations matched on alternate journal names: 0
Final total matches 2

 181 BELENGUER JM, 2003, COMPUT OPER RES
Full citation mathces 0
Potential citations not immediately matched: 0
Citations matched on alternate journal names: 0
Final total matches 0

 182 GREIST

In [239]:
print(len(new_citations))
d['CR_matched'] = new_citations

192


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [243]:
d.to_csv('../data/arp_grp_2010_2019_references_matched.csv', index=False)
print(len(d))

192


In [244]:
d

,Unnamed: 0,AU,TI,SO,JI,AB,DE,ID,LA,DT,...,GA,BO,RP,DB,AU_UN,AU1_UN,AU_UN_NR,SR_FULL,SR,CR_matched
1,"WILLEMSE EJ, 2019, COMPUT OPER RES",WILLEMSE EJ;JOUBERT JW,EFFICIENT LOCAL SEARCH STRATEGIES FOR THE MIXE...,COMPUTERS \& OPERATIONS RESEARCH,COMPUT. OPER. RES.,"IN THIS PAPER, WE EXTEND OUR PREVIOUS WORK ON ...",WASTE MANAGEMENT; CAPACITATED ARC ROUTING PROB...,NaN,ENGLISH,ARTICLE,...,HO2BL,NaN,"WILLEMSE, EJ (REPRINT AUTHOR), UNIV PRETORIA, ...",ISI,UNIV PRETORIA,UNIV PRETORIA,NaN,"WILLEMSE EJ, 2019, COMPUT OPER RES","WILLEMSE EJ, 2019, COMPUT OPER RES","BELENGUER JM, 2006, COMPUT OPER RES;BEULLENS P..."
2,"ARAKAKI RK, 2019, COMPUT OPER RES",ARAKAKI RK;USBERTI FL,AN EFFICIENCY-BASED PATH-SCANNING HEURISTIC FO...,COMPUTERS \& OPERATIONS RESEARCH,COMPUT. OPER. RES.,THE CAPACITATED ARC ROUTING PROBLEM (CARP) IS ...,CAPACITATED ARC ROUTING PROBLEM; CONSTRUCTIVE ...,ALGORITHMS; OPTIMIZATION; SEARCH; TIME,ENGLISH,ARTICLE,...,HI9EB,NaN,"ARAKAKI, RK (REPRINT AUTHOR), UNIV ESTADUAL CA...",ISI,UNIV ESTADUAL CAMPINAS,UNIV ESTADUAL CAMPINAS,NaN,"ARAKAKI RK, 2019, COMPUT OPER RES","ARAKAKI RK, 2019, COMPUT OPER RES","ARAKAKI RK, 2018, COMPUT OPER RES;BELENGUER JM..."
3,"DE ARMAS J, 2019, ANN OPER RES",DE ARMAS J;KEENAN P;JUAN AA;MCGARRAGHY S,SOLVING LARGE-SCALE TIME CAPACITATED ARC ROUTI...,ANNALS OF OPERATIONS RESEARCH,ANN. OPER. RES.,THIS PAPER DISCUSSES THE TIME CAPACITATED ARC ...,CAPACITATED ARC ROUTING PROBLEM; TIME-BASED CA...,TABU SEARCH ALGORITHM; ITERATED LOCAL SEARCH; ...,ENGLISH,ARTICLE,...,HI3XH,NaN,"JUAN, AA (REPRINT AUTHOR), OPEN UNIV CATALONIA...",ISI,UNIV POMPEU FABRA;UNIV COLL DUBLIN;OPEN UNIV C...,OPEN UNIV CATALONIA,NaN,"DE ARMAS J, 2019, ANN OPER RES","DE ARMAS J, 2019, ANN OPER RES","AMBERG A, 2000, EUR J OPER RES;BALDACCI R, 200..."
5,"TFAILI S, 2019, RAIRO-OPER RES",TFAILI S;DKHIL H;SBIHI A;YASSINE A,EFFICIENT ALGORITHMS UNDER DYNAMIC GRAPHS TO S...,RAIRO-OPERATIONS RESEARCH,RAIRO-OPER. RES.,IN THIS PAPER WE DEVELOP SEVERAL APPROACHES TO...,ROUTING PROBLEMS; GRAPH THEORY; SPARSITY; TABU...,CUTTING PLANE ALGORITHM; CUT ALGORITHM; BRANCH...,ENGLISH,ARTICLE,...,HN7MX,NaN,"SBIHI, A (REPRINT AUTHOR), UNIV LE HAVRE NORMA...",ISI,UNIV LE HAVRE NORMANDIE;BREST BUSINESS SCH;UNI...,UNIV LE HAVRE NORMANDIE,NaN,"TFAILI S, 2019, RAIRO-OPER RES","TFAILI S, 2019, RAIRO-OPER RES","BELENGUER JM, 2006, COMPUT OPER RES;BELENGUER ..."
6,"MOFID-NAKHAEE E, 2019, WASTE MANAGE RES",MOFID NAKHAEE E;BARZINPOUR F,A MULTI-COMPARTMENT CAPACITATED ARC ROUTING PR...,WASTE MANAGEMENT \& RESEARCH,WASTE MANAGE. RES.,MUNICIPAL SOLID WASTE COLLECTION IS AN INCREAS...,ARC ROUTING PROBLEM; MULTI-COMPARTMENT VEHICLE...,ANT COLONY ALGORITHM; TIME RESTRICTIONS; OPTIM...,ENGLISH,ARTICLE,...,HH0FJ,NaN,"BARZINPOUR, F (REPRINT AUTHOR), IRAN UNIV SCI ...",ISI,IRAN UNIV SCI AND TECHNOL,IRAN UNIV SCI AND TECHNOL,NaN,"MOFID-NAKHAEE E, 2019, WASTE MANAGE RES","MOFID-NAKHAEE E, 2019, WASTE MANAGE RES","AMPONSAH SK, 2004, WASTE MANAGE;BAUTISTA J, 20..."
7,"FENG L, 2019, EVOLUTIONARY AND SWARM INTELLIGE...",FENG L;ONG YS;GUPTA A,GENETIC ALGORITHM AND ITS ADVANCES IN EMBRACIN...,EVOLUTIONARY AND SWARM INTELLIGENCE ALGORITHMS,EVOLUTIONARY AND SWARM INTELLIGENCE ALGORITHMS,A GENETIC ALGORITHM (GA) IS A STOCHASTIC SEARC...,EVOLUTIONARY OPTIMIZATION; GENETIC ALGORITHM; ...,OPTIMIZATION; DESIGN,ENGLISH,"ARTICLE, BOOK CHAPTER",...,BK9FI,EVOLUTIONARY AND SWARM INTELLIGENCE ALGORITHMS,"FENG, L (REPRINT AUTHOR), NANYANG TECHNOL UNIV...",ISI,NANYANG TECHNOL UNIV;CHONGQING UNIV,NANYANG TECHNOL UNIV,NaN,"FENG L, 2019, EVOLUTIONARY AND SWARM INTELLIGE...","FENG L, 2019, EVOLUTIONARY AND SWARM INTELLIGE...","FENG L, 2010, 2010 IEEE CONGRESS ON EVOLUTIONA..."
9,"LI M, 2018, COMPUT IND ENG",LI M;ZHEN L;WANG S;LV W;QU X,UNMANNED AERIAL VEHICLE SCHEDULING PROBLEM FOR...,COMPUTERS \& INDUSTRIAL ENGINEERING,COMPUT. IND. ENG.,FOR MORE ACCURATE MULTIPLE-PERIOD REAL-TIME MO...,UAV ROUTING PROBLEM; ARC ROUTING PROBLEM; INVE...,INVENTORY-ROUTING PROBLEM; ARC; ALGORITHM; LOC...,ENGLISH,ARTICLE